In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time 

cap= cv2.VideoCapture(0)
pose= mp.solutions.pose
pose_o= pose.Pose()
drawing = mp.solutions.drawing_utils
color=(255,0,255)


#initialize some variables for setting up and down condition in an elbow case
status= "Down" # Determines the status whenever we lift the dumbells
count=0 # a counter

#To calculate the angle between shoulder, elbow and wrist, because of this I have written a function calc_angle
def calc_angle(a,b,c):
    global angle
    global bar
#a,b,c are nothing but a reference for shoulder, elbow, wrist points. Also a,b,c are wriiten in an array format for x and y axis
    a= np.array(a) 
    b= np.array(b)
    c= np.array(c)
    rad = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle= np.abs(rad*180/np.pi)
    bar= np.interp(angle, (150,205), (400,50))

    
    if angle>180:
        angle= 360-angle
    
    return angle





while True:
    ret, frame= cap.read()
    frame= cv2.flip(frame, 1)
    res= pose_o.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # converting the bgr to rgb 
    
    if res.pose_landmarks:
        finalres=res.pose_landmarks.landmark 
        #print(finalres)
        #Angle Retrieval for the elbow and also analyzing it's movements
        shoulder= [finalres[pose.PoseLandmark.LEFT_SHOULDER.value].x, finalres[pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow= [finalres[pose.PoseLandmark.LEFT_ELBOW.value].x, finalres[pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist= [finalres[pose.PoseLandmark.LEFT_WRIST.value].x, finalres[pose.PoseLandmark.LEFT_WRIST.value].y]
        angle= calc_angle(shoulder, elbow, wrist)
        cv2.putText(frame, str(int(angle)), tuple(np.multiply(elbow, [640,280]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 1, (150,255,55),2, cv2.LINE_AA)
        
        drawing.draw_landmarks(frame, res.pose_landmarks, pose.POSE_CONNECTIONS,
                              drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=6),
                               drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))
        #Drawing colors are specified using DrawingSpec for the landmarks as well as the dots
        
        if angle>170:
            status= "Down"
            cv2.putText(frame, "0%", (500,45),cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0),2, cv2.LINE_AA )
            color=(0,255,0)           
        
        if angle<48 and status=="Down":
            status= "Up"
            count+= 1
            print(count)
            #print("The Percentage is :",per)
            cv2.putText(frame, "100%", (500,45),cv2.FONT_HERSHEY_SIMPLEX,1, (255,0,0),2, cv2.LINE_AA )
            color=(255,0,0)
            
        
    cv2.rectangle(frame, (0,0), (275,73), (245,117,66), -1)
    
    #Displaying the number of reps or counts 
    cv2.putText(frame, "No of Reps:", (10,15),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0),2,cv2.LINE_AA)
    cv2.putText(frame, str(count), (10,50),cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,255),2, cv2.LINE_AA)
    
    #Displaying the status of the elbow
    
    cv2.putText(frame, "Status:", (150,15),cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,0),2, cv2.LINE_AA )
    
    cv2.putText(frame, status, (150,50),cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,255),2, cv2.LINE_AA )
    
    #Bar status at the side of the video to show an effect
    
    cv2.rectangle(frame, (570,50), (600,400), color, 3)
    cv2.rectangle(frame, (570,int(bar)), (600,400), color, cv2.FILLED)
    
    
        
    
    cv2.imshow("Pose", frame)
    
    if cv2.waitKey(50) & 0xff== ord('q'):
            break
cap.release()
cv2.destroyAllWindows()